# Segmentace obrazu
Cvičení je zaměřené na správné využití osvětlení při nasvícení objektu a následné využití metod pro segmentaci obrazu.

### Import knihoven a konfigurace

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import cv2
from pypylon import pylon 

%matplotlib inline

### Pomocné funkce
Z následujících funkcí je potřeba vybírat ty vhodné pro splnění úkolu.

Funkce k připojení kamery.

In [ ]:
CAM_WIDTH = 1080
CAM_HEIGHT = 720

# List of features to create widgets
FEATURES = [
    {
        "name": "GainRaw",
        "type": "int"
    },
    {
        "name": "ExposureTimeRaw",
        "type": "int",
        "max": 35000,
        "min": 35,
        "step": 140,
        "value": 3500
    },
    {
        "name": "Height",
        "type": "int_text",
        "max": 1200,
        "min": 100,
        "step": "5"
    },
    {
        "name": "Width",
        "type": "int_text",
        "max": 1920,
        "min": 100,
        "step": "5"
    }
]

def connect_camera(serial_number):
    ''' Connects camera specified with its serial number
    
    Parameters
    ----------
    serial_number : string
        Camera's serial number.
    grabbed_images_path : string
        Path to folder where the saved images will be stored.
    Returns
    -------
    camera : object
    '''
    info = None
    for i in pylon.TlFactory.GetInstance().EnumerateDevices():
        if i.GetSerialNumber() == serial_number:
            info = i
            break
    else:
        print('Camera with {} serial number not found'.format(serial_number))

    # VERY IMPORTANT STEP! To use Basler PyPylon OpenCV viewer you have to call .Open() method on you camera
    if info is not None:
        camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateDevice(info)) 
        camera.Open()
        return camera
    else:
        return None    

Funkce pro zobrazení okna s přidáním eventu na klik myší.

In [ ]:
def print_xy(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print('x = %d, y = %d'% (x, y))
        
def show_camera_window(img, scale=1):
    window_name = 'Camera capture (' + serial_number + ')'
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
    cv2.resizeWindow(window_name, img.shape[1] * scale, img.shape[0] * scale)
    cv2.setMouseCallback(window_name, print_xy)
    cv2.imshow(window_name, img)

Metody předzpracování.

In [ ]:
def to_gray(img):
    ''' Converts image to monochrome
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Ouput image.
    '''
    dst = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return dst

def negative(img):
    ''' Converts image to its negative
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Ouput image.
    '''
    dst = 255 - img
    return dst

def crop(img, tl_x, tl_y, br_x, br_y):
    ''' Crops image by added coordinates
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    tl_x : int
        TOP-LEFT corner's x-coordinate
    tl_y : int
        TOP-LEFT corner's y-coordinate
    br_x : int
        BOTTOM-RIGHT corner's x-coordinate
    br_y : int
        BOTTOM-RIGHT corner's y-coordinate
    Returns
    -------
    Ouput image.
    '''
    roi = img[tl_y:br_y, tl_x:br_x]
    return roi    

Metody filtrace.

In [ ]:
def filtration_median(img, filter_size):
    '''Filters image noise using median algorithm
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    filter_size : int
        Size of median filter.
    Returns
    -------
    Ouput image.
    '''
    dst = cv2.medianBlur(img, filter_size)
    return dst    

Metody segmentace.

In [ ]:
def segmentation_one_threshold(img, threshold):
    '''Segments image into black & white using one threshold
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    threshold : int
        Pixels with value lower than threshold are considered black, the others white.
    Returns
    -------
    Ouput image.
    '''
    ret, dst = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)
    return dst

def segmentation_auto_threshold(img):
    '''Segments image into black & white using automatic threshold
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Ouput image.
    '''
    ret, dst = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return dst

def segmentation_two_thresholds(img, lower, higher):
    '''Segments image into black & white using two thresholds
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    lower : int
        Pixels with value lower than threshold are considered black, the others white.
    higher : int
        Pixels with value higher than threshold are considered black, the others white.
    Returns
    -------
    Ouput image.
    '''
    dst = cv2.inRange(img, min(lower, higher), max(lower, higher))
    return dst

Metoda pro zobrazení různého množství obrázků.

In [ ]:
def show_images(*imgs, scale=1, window_name='Image preview'):
    """ Opens multiple image previews depending on the length of the input *imgs list.
    The preview is terminated by pressing the 'q' key.
    
    Parameters
    ----------
    *imgs : list
        Multiple input images which have to be shown.
    scale : double
        Scale of shown image window.
    window_name : Optional[string]
        An optional window name.
    Returns
    -------
    None
    """
    for i, img in enumerate(imgs, 1):
        window_name_id = window_name + ' ' + str(i)
        cv2.namedWindow(window_name_id, cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
        cv2.resizeWindow(window_name_id, img.shape[1] * scale, img.shape[0] * scale)
        cv2.moveWindow(window_name_id, 0, i*10)

    while 1:
        for i, img in enumerate(imgs, 1):
            cv2.imshow(window_name + ' ' + str(i), img)
            
        k = cv2.waitKey(0)

        if k == ord('q'):
            break

    cv2.destroyAllWindows()

Metoda pro OCR obrazu

In [ ]:
def ocr(img):
    return 'FIT'

---

Seznam funkcí pro přehlednost:
- `connect_camera(...)`
- `show_images(...)`


- `to_gray(...)`
- `negative(...)`
- `crop(...)`
- `filtration_median(...)`
- `segmentation_one_threshold(...)`
- `segmentation_auto_threshold(...)`
- `segmentation_two_thresholds(...)`

### Úkol
Zvolte vhodné funkce pro segmentaci obrazu a přečtěte text na obrazu snímaném kamerou.

Pro testování algoritmu využijte funkci `impro()`. Po změně obsahu funkce je potřeba načíst znovu i blok pro spuštění snímání z kamery.

Po vyladění algoritmu, využijte obrázek z kamery a nechte ho přečíst OCR. V případě, že výsledek bude po porovnání `True`, úkol jste splnili.

#### 1) Nastavte správně kameru.

In [ ]:
serial_number = '22018112' ### vlastní sériové číslo kamery
width = 1280
height = 720

camera = connect_camera(serial_number)

#### 2) Definujte obsah funkce pro zpracování obrazu, konkrétně jeho segmentaci.

In [ ]:
def impro(img):
    img = to_gray(img)
    img = crop(img, 713, 224, 822, 328)
    #img = segmentation_one_threshold(img, 50)
    img = segmentation_auto_threshold(img)
    #img = segmentation_two_thresholds(img, 20, 80)
    #img = negative(img)
    #img = filtration_median(img, 3)
    show_camera_window(img, 3)

#### 3) Spusťte snímání a testujte algoritmus pro zpracování obrazu.

Info k oknu:
- Pro spuštění okna kamery je zapotřebí kliknout na tlačítko `Run Interact`. 
- Okno kamery se vypne stisknutím tlačítka `q`.
- Po kliku levého tlačítka myši se vytisknou souřadnice místa v obraze.
- Pro projevení změn v nastavení pomocí GUI prvků je nutné ukončit okno kamery a znovu ho spustit.

In [ ]:
from pypylon_opencv_viewer import BaslerOpenCVViewer
    
viewer = BaslerOpenCVViewer(camera)
viewer.set_features(FEATURES)
viewer.set_impro_function(impro)
viewer.run_interaction_continuous_shot(window_size=(width, height))

---

#### 4) Doplňte funkční verzi algoritmu.

In [ ]:
def algorithm(img):
    img = to_gray(img)
    img = crop(img, 713, 224, 822, 328)
    img = segmentation_auto_threshold(img)
    return img

#### 5) Využijte funkční algoritmus na obrázek z kamery.

In [ ]:
viewer = BaslerOpenCVViewer(camera)
image = viewer.get_image()
image = algorithm(image)

text = ocr(image)
print('Přečtený text je: ' + text) 

#### 6) Zkontrolujte správnost

In [ ]:
ref_text = 'FIT'
if text == ref_text:
    print('Úkol jste splnili!')
else:
    print('Úkol je třeba dál ladit ...')